In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("movies.csv")

In [ ]:
df.info()

In [ ]:
df["title"] = df["MOVIES"].str.strip()

In [ ]:
def extract_directors_stars(text):
    text = text.replace("\n", "")

    if "Director" in text and "Star" in text:
        parts = text.split("|")
        director = parts[0].split(":")[1].strip().split(", ")
        stars = parts[1].split(":")[1].strip().split(", ")
    elif "Star" in text:
        director = None
        stars = text.split(":")[1].strip().split(", ")
    elif "Director" in text:
        director = text.split(":")[1].strip().split(", ")
        stars = None
    else:
        director = None
        stars = None
        # print(text)

    # if director is None or stars is None:
    #     print(text)
    return director, stars

In [ ]:
df["directors"], df["stars"] = zip(*df["STARS"].apply(extract_directors_stars))

In [ ]:
import re


def extract_numbers(text):
    if isinstance(text, str):
        numbers = re.findall(r"\d+", text)
        return [int(num) for num in numbers]
    else:
        return None

In [ ]:
df["year"] = df["YEAR"].apply(extract_numbers)

In [ ]:
get_first = lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None
get_second = lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None

df["year_start"] = df["year"].apply(get_first)
df["year_end"] = df["year"].apply(get_second)

In [ ]:
df["genres"] = df["GENRE"].apply(
    lambda x: x.replace("\n", "").strip().split(", ") if isinstance(x, str) else None
)

In [ ]:
df["votes"] = pd.to_numeric(df["VOTES"].str.replace(",", ""), errors="coerce")

In [ ]:
df["gross"] = pd.to_numeric(df["Gross"].str.strip("$M"), errors="coerce")

In [ ]:
for i in df.Gross.unique():
    print(i)

In [ ]:
from sqlalchemy import create_engine

db_params = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "postgres",
    "host": "localhost",  # Change to your database host
    "port": "5432",  # Change to your database port
}
db_url = f"postgresql://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"
engine = create_engine(db_url)

In [ ]:
df_final = df.rename(
    columns={"ONE-LINE": "one_line", "RATING": "rating", "RunTime": "run_time"}
)[
    [
        "title",
        "year_start",
        "year_end",
        "genres",
        "rating",
        "one_line",
        "directors",
        "stars",
        "votes",
        "run_time",
        "gross",
    ]
]

In [ ]:
# df_final.to_sql(name='movies', con=engine, schema='public', if_exists='replace', method='multi', index=False)

In [ ]:
df = df_final

# Normalize

In [ ]:
def explode_list(df, col_name):
    df_temp = df[col_name].explode().reset_index(drop=True)
    df_temp.rename(columns={col_name: "name"}, inplace=True)
    return df_temp

In [ ]:
directors_df = df["directors"].explode().reset_index(drop=True).unique()

stars_df = df["stars"].explode().reset_index(drop=True).unique()

genres_df = df["genres"].explode().reset_index(drop=True).unique()

In [ ]:
import psycopg2

conn = psycopg2.connect(
    database="postgres", user="postgres", password="postgres", host="localhost"
)
cur = conn.cursor()

In [ ]:
# # Define table creation statements (assuming tables don't exist)
# create_movies_table = """
# CREATE TABLE IF NOT EXISTS movies (
#   movie_id SERIAL PRIMARY KEY,
#   title VARCHAR ,
#   year_start INTEGER,
#   year_end INTEGER,
#   rating FLOAT,
#   one_line TEXT
# );
# """

# create_directors_table = """
# CREATE TABLE IF NOT EXISTS directors (
#   director_id SERIAL PRIMARY KEY,
#   name VARCHAR
# );
# """

# create_stars_table = """
# CREATE TABLE IF NOT EXISTS stars (
#   star_id SERIAL PRIMARY KEY,
#   name VARCHAR
# );
# """

# create_genres_table = """
# CREATE TABLE IF NOT EXISTS genres (
#   genre_id SERIAL PRIMARY KEY,
#   name VARCHAR
# );
# """

# # Execute the CREATE TABLE statements you defined earlier
# cur.execute(create_directors_table)
# cur.execute(create_stars_table)
# cur.execute(create_genres_table)
# cur.execute(create_movies_table)
# conn.commit()

In [ ]:
def insert_unique_col(lst, tbl_name):
    for value in lst:
        if value is not None:
            cur.execute(
                f"INSERT INTO {tbl_name} (name) VALUES (%s) ON CONFLICT DO NOTHING;",
                (value,),
            )
            conn.commit()

In [ ]:
# insert_unique_col(directors_df, 'directors')
# insert_unique_col(stars_df, 'stars')
# insert_unique_col(genres_df, 'genres')

In [ ]:
# df.explode('genres')

# Option 2

In [ ]:
df["movie_id"] = df.index

In [ ]:
movie_genre_df

In [ ]:
## Normalize Genres
movie_genre_df = df.explode("genres")[["movie_id", "genres"]]
genres_df = pd.DataFrame(
    movie_genre_df["genres"].explode().unique(), columns=["genre_name"]
)
genres_df["genre_id"] = genres_df.index
movie_genre_df = movie_genre_df.merge(
    genres_df, left_on="genres", right_on="genre_name", how="left"
)[["movie_id", "genre_id"]]

## Normalize Directors
movie_director_df = df.explode("directors")[["movie_id", "directors"]]
directors_df = pd.DataFrame(
    movie_director_df["directors"].explode().unique(), columns=["director_name"]
)
directors_df["director_id"] = directors_df.index
movie_director_df = movie_director_df.merge(
    directors_df, left_on="directors", right_on="director_name", how="left"
)[["movie_id", "director_id"]]

## Normalize Stars (Actors)
movie_star_df = df.explode("stars")[["movie_id", "stars"]]
stars_df = pd.DataFrame(
    movie_star_df["stars"].explode().unique(), columns=["actor_name"]
)
stars_df["actor_id"] = stars_df.index
movie_star_df = movie_star_df.merge(
    stars_df, left_on="stars", right_on="actor_name", how="left"
)[["movie_id", "actor_id"]]

In [ ]:
movies_df = df.drop(columns=["directors", "stars", "genres"])

In [ ]:
movies_df.to_sql(
    name="movies",
    con=engine,
    schema="public",
    if_exists="replace",
    method="multi",
    index=False,
)
movies_df.to_sql("movies", engine, if_exists="replace", index=False)
genres_df.to_sql("genres", engine, if_exists="replace", index=False)
directors_df.to_sql("directors", engine, if_exists="replace", index=False)
stars_df.to_sql("stars", engine, if_exists="replace", index=False)
movie_genre_df.to_sql("movie_genres", engine, if_exists="replace", index=False)
movie_director_df.to_sql("movie_directors", engine, if_exists="replace", index=False)
movie_star_df.to_sql("movie_stars", engine, if_exists="replace", index=False)

In [112]:
!jupyter nbconvert --to script main.ipynb